In [203]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
import json
from fake_useragent import UserAgent
import random
import time

newt_url_front = 'https://roadsafety.tw/api/DashboardAjax/GetDashboardStaticData?sCyear='

newt_url_mid  = '%E5%B9%B4&sM='

newt_url_tail = '%E6%9C%88&sCity=%E6%96%B0%E5%8C%97%E5%B8%82&Type=%E8%88%87%E5%8E%BB%E5%B9%B4%E5%90%8C%E6%9C%9F%E6%AF%94%E8%BC%83&TypeDeathHurt=0'


taipei_url_front = 'https://roadsafety.tw/api/DashboardAjax/GetDashboardStaticData?sCyear='

taipei_url_mid = '%E5%B9%B4&sM='

taipei_url_tail = '%E6%9C%88&sCity=%E8%87%BA%E5%8C%97%E5%B8%82&Type=%E8%88%87%E5%8E%BB%E5%B9%B4%E5%90%8C%E6%9C%9F%E6%AF%94%E8%BC%83&TypeDeathHurt=0'

years_list = [
    '098','099','100','101','102','103','104','105','106','107','108','109'
]

month_list = [
    '01','02','03','04','05','06','07','08','09','10','11','12'
]

def newtaipei_post(year , month , filename):
    global col_list
    global name
    header = {
    'Referer': 'https://roadsafety.tw/Dashboard/Custom?type=%E7%B5%B1%E8%A8%88%E5%BF%AB%E8%A6%BD',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'
    }

    data = {
        'sCyear': str(year) + '年',
        'sM': str(month) + '月',
        'sCity': '新北市',
        'Type': '與去年同期比較',
        'TypeDeathHurt': '0'
    }


    r = requests.get(newt_url_front + str(year) + newt_url_mid +str(month) + newt_url_tail , headers = header , data = data )
    versionInfo = r.text
    versionInfoPython = json.loads(versionInfo)
    print(versionInfoPython)
    
    col_list = []
    num = []
    reason = []
    name = []
 
    for i in range(len(versionInfoPython)):
        col_list.append(versionInfoPython[i].get('col'))
        name.append(versionInfoPython[i].get('col'))
        num.append(versionInfoPython[i].get('qty'))
        reason.append(versionInfoPython[i].get('tooltip'))
        
        

    col_list = pd.DataFrame(col_list , columns = ['Title'])
    col_list['新北市' + filename] = num
    col_list = col_list.T
    return col_list


def taipei_post(year , month , filename):
    global col_list
    global name
    header = {
    'Referer': 'https://roadsafety.tw/Dashboard/Custom?type=%E7%B5%B1%E8%A8%88%E5%BF%AB%E8%A6%BD',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'
    }

    data = {
        'sCyear': str(year) + '年',
        'sM': str(month) + '月',
        'sCity': '臺北市',
        'Type': '與去年同期比較',
        'TypeDeathHurt': '0'
    }


    r = requests.get(taipei_url_front + str(year) + taipei_url_mid +str(month) + taipei_url_tail , headers = header , data = data )
    versionInfo = r.text
    versionInfoPython = json.loads(versionInfo)
    print(versionInfoPython)
    
    col_list = []
    num = []
    reason = []
    name = []
 
    for i in range(len(versionInfoPython)):
        col_list.append(versionInfoPython[i].get('col'))
        name.append(versionInfoPython[i].get('col'))
        num.append(versionInfoPython[i].get('qty'))
        reason.append(versionInfoPython[i].get('tooltip'))
        
        

    col_list = pd.DataFrame(col_list , columns = ['Title'])
    col_list['臺北市' + filename] = num
    col_list = col_list.T
    return col_list

In [204]:
new_all_df = pd.DataFrame()

for i in years_list: 
    new_df = []
    for j in month_list:
        t = newtaipei_post(i , j , i + j)
        if len(new_df) == 0:
            new_df = t
        else:
            new_df = pd.concat([t , new_df.drop('Title')])
            
    if len(new_all_df) == 0:
        new_all_df = new_df
    else:
        new_all_df = pd.concat([new_all_df , new_df.drop('Title')])
        
taipei_all_df = pd.DataFrame()

for i in years_list: 
    taipei_df = []
    for j in month_list:
        t = taipei_post(i , j , i + j)
        if len(taipei_df) == 0:
            taipei_df = t
        else:
            taipei_df = pd.concat([t , taipei_df.drop('Title')])
            
    if len(taipei_all_df) == 0:
        taipei_all_df = taipei_df
    else:
        taipei_all_df = pd.concat([taipei_all_df , taipei_df.drop('Title')])
        
taipei_all_df = taipei_all_df.drop(['臺北市10910','臺北市10911','臺北市10912'])
new_all_df = new_all_df.drop(['新北市10910','新北市10911','新北市10912'])

final_df = pd.concat([taipei_all_df , new_all_df.drop('Title')])
final_df

[{'row': ['098年', '01月', '新北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': -7.6190476190476195, 'descriptionHtml': ' 1,746 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-144&lt;/span&gt;', 'qty': 1746.0, 'diff': '-144', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['098年', '01月', '新北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': 100.0, 'descriptionHtml': ' 34 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span class=&#x27;number-plus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle&#x27;&gt;&lt;/i&gt;&lt;/span&gt;&#x2B;17&lt;/span&gt;', 'qty': 34.0, 'diff': '17', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['098年', '01月', '新北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -99999999.0, 'descriptionHtml': ' 0 &amp;nbsp;&lt;span style=&#x27;color:#888888;&#x27;&gt;&lt;i class=&#x2

[{'row': ['098年', '02月', '新北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': 0.7534659433393611, 'descriptionHtml': ' 3,343 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span class=&#x27;number-plus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle&#x27;&gt;&lt;/i&gt;&lt;/span&gt;&#x2B;25&lt;/span&gt;', 'qty': 3343.0, 'diff': '25', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['098年', '02月', '新北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': 81.81818181818183, 'descriptionHtml': ' 60 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span class=&#x27;number-plus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle&#x27;&gt;&lt;/i&gt;&lt;/span&gt;&#x2B;27&lt;/span&gt;', 'qty': 60.0, 'diff': '27', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['098年', '02月', '新北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -99999999.0, 'descriptionHtml': ' 0 &amp;nbsp;&lt;span style=&#x27;color:#888888;&#x27;&gt;&lt;i class=&#x27;f

[{'row': ['098年', '03月', '新北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': -0.28254288597376387, 'descriptionHtml': ' 4,941 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-14&lt;/span&gt;', 'qty': 4941.0, 'diff': '-14', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['098年', '03月', '新北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': 39.62264150943396, 'descriptionHtml': ' 74 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span class=&#x27;number-plus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle&#x27;&gt;&lt;/i&gt;&lt;/span&gt;&#x2B;21&lt;/span&gt;', 'qty': 74.0, 'diff': '21', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['098年', '03月', '新北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -99999999.0, 'descriptionHtml': ' 1 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span

[{'row': ['098年', '04月', '新北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': -1.2165450121654502, 'descriptionHtml': ' 6,496 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-80&lt;/span&gt;', 'qty': 6496.0, 'diff': '-80', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['098年', '04月', '新北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': 28.000000000000004, 'descriptionHtml': ' 96 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span class=&#x27;number-plus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle&#x27;&gt;&lt;/i&gt;&lt;/span&gt;&#x2B;21&lt;/span&gt;', 'qty': 96.0, 'diff': '21', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['098年', '04月', '新北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -99999999.0, 'descriptionHtml': ' 1 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span

[{'row': ['098年', '05月', '新北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': -2.497918401332223, 'descriptionHtml': ' 8,197 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-210&lt;/span&gt;', 'qty': 8197.0, 'diff': '-210', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['098年', '05月', '新北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': 3.9215686274509802, 'descriptionHtml': ' 106 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span class=&#x27;number-plus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle&#x27;&gt;&lt;/i&gt;&lt;/span&gt;&#x2B;4&lt;/span&gt;', 'qty': 106.0, 'diff': '4', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['098年', '05月', '新北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -99999999.0, 'descriptionHtml': ' 1 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;spa

...

[{'row': ['109年', '09月', '臺北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': 10.438018545632016, 'descriptionHtml': ' 18,103 &amp;nbsp;&lt;span class=&#x27;text-danger&#x27;&gt;&lt;span class=&#x27;number-plus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle&#x27;&gt;&lt;/i&gt;&lt;/span&gt;&#x2B;1,711&lt;/span&gt;', 'qty': 18103.0, 'diff': '1,711', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['109年', '09月', '臺北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': -27.450980392156865, 'descriptionHtml': ' 74 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-28&lt;/span&gt;', 'qty': 74.0, 'diff': '-28', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['109年', '09月', '臺北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -100.0, 'descriptionHtml': ' 0 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;

[{'row': ['109年', '10月', '臺北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': -1.7476255088195387, 'descriptionHtml': ' 18,103 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-322&lt;/span&gt;', 'qty': 18103.0, 'diff': '-322', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['109年', '10月', '臺北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': -33.33333333333333, 'descriptionHtml': ' 74 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-37&lt;/span&gt;', 'qty': 74.0, 'diff': '-37', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['109年', '10月', '臺北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -100.0, 'descriptionHtml': ' 0 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;

[{'row': ['109年', '11月', '臺北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': -12.117093062770037, 'descriptionHtml': ' 18,103 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-2,496&lt;/span&gt;', 'qty': 18103.0, 'diff': '-2,496', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['109年', '11月', '臺北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': -39.34426229508197, 'descriptionHtml': ' 74 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-48&lt;/span&gt;', 'qty': 74.0, 'diff': '-48', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['109年', '11月', '臺北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -100.0, 'descriptionHtml': ' 0 &amp;nbsp;&lt;span class=&#x27;text-primary&#

[{'row': ['109年', '12月', '臺北市', '與去年同期比較', '0'], 'col': ['事故總件數'], 'colCode': ['01'], 'value': -20.88541211432567, 'descriptionHtml': ' 18,103 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-4,779&lt;/span&gt;', 'qty': 18103.0, 'diff': '-4,779', 'tooltip': '警政署交通事故案件數(A1死亡案件+A2受傷案件)合計值'}, {'row': ['109年', '12月', '臺北市', '與去年同期比較', '0'], 'col': ['30日死亡'], 'colCode': ['02'], 'value': -43.51145038167939, 'descriptionHtml': ' 74 &amp;nbsp;&lt;span class=&#x27;text-primary&#x27;&gt;&lt;span class=&#x27;number-minus&#x27;&gt;&lt;i class=&#x27;fas fa-xs fa-triangle fa-rotate-180&#x27;&gt;&lt;/i&gt;&lt;/span&gt;-57&lt;/span&gt;', 'qty': 74.0, 'diff': '-57', 'tooltip': '發生交通事故30日內死亡的人數總計'}, {'row': ['109年', '12月', '臺北市', '與去年同期比較', '0'], 'col': ['兒童(0-12歲)'], 'colCode': ['07'], 'value': -100.0, 'descriptionHtml': ' 0 &amp;nbsp;&lt;span class=&#x27;text-primary&#x

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
Title,事故總件數,30日死亡,兒童(0-12歲),少年(13-17歲),少年無照機車騎士<br>(含大重機),少年自行車騎士<br>(含電自、電輔),年輕人(18-24歲),年輕人機車騎士<br>(含大重機),高齡者<br/>(65歲以上),高齡者機車騎士<br>(含大重機),...,外籍人士,酒駕案件導致死亡,大型車造成機車涉入<br>(含大重機),大型車造成行人涉入<br>(含代步),爆胎案件導致死亡,大型車造成自行車<br>涉入(含電自、電輔),平交道事故總件數,平交道事故導致死亡,動力機械案件<br>導致死亡,營業用小客車案件<br>導致死亡
臺北市09812,15326,133,0,2,0,0,16,14,44,10,...,0,6,9,5,0,1,0,0,0,14
臺北市09811,13988,124,0,2,0,0,15,13,40,9,...,0,6,8,5,0,1,0,0,0,12
臺北市09810,12543,108,0,2,0,0,14,12,35,9,...,0,5,8,4,0,1,0,0,0,10
臺北市09809,11357,99,0,2,0,0,14,12,31,8,...,0,5,7,4,0,1,0,0,0,8
臺北市09808,10199,91,0,2,0,0,14,12,29,8,...,0,4,7,4,0,1,0,0,0,8
臺北市09807,8946,79,0,2,0,0,14,12,26,7,...,0,3,7,4,0,1,0,0,0,7
臺北市09806,7430,64,0,2,0,0,13,11,20,5,...,0,2,5,4,0,1,0,0,0,6
臺北市09805,6073,57,0,2,0,0,12,10,20,5,...,0,2,3,3,0,0,0,0,0,6
臺北市09804,4704,46,0,2,0,0,11,9,16,5,...,0,2,2,2,0,0,0,0,0,5


In [206]:
final_df.to_csv('交通事故統計_雙北.csv' , encoding = 'utf-8')